In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import os
import gzip

import numpy as np
from IPython import display

import time

from keras import backend as K
from keras.callbacks import Callback
from keras.models import Model, load_model, model_from_json
from keras.optimizers import SGD, Adam
from keras.utils import HDF5Matrix
from keras.applications.vgg16 import VGG16
from keras.layers import Dense, Dropout, GlobalAveragePooling2D, GlobalMaxPooling2D, Flatten, Concatenate 
# Conv2D, Input, Flatten, MaxPooling2D, UpSampling2D, concatenate, Cropping2D, Reshape, BatchNormalization
from keras.applications.vgg19 import VGG19

from keras.models import load_model
import matplotlib.pyplot as plt
%matplotlib inline


import tensorflow as tf

Using TensorFlow backend.


In [3]:
import sys
sys.path.append("..")

In [4]:
from utils.load_data import load_data
from utils.preprocess import DataGenerator
from utils.comparams import calculate_auc, auc
from utils.callbacks import PlotCurves

# DATA

In [ ]:
def run_test(model_path, data_dir='../data/macenko/', epoch=15, batch_size=128):
    
    x_test, y_test_true = load_data(data_dir, purpose='test', norm='macenko')
    
    # indexes
    test_id = np.arange(len(x_test))

    partition = {}
    partition['test'] = test_id

    test_labels = {str(i) : y_test_true[i].flatten()[0] for i in test_id}

    # Parameters for generators
    params = {
        'dim': (96, 96),
        'batch_size': batch_size,
        'n_classes': 2,
        'aug': False,
        'shuffle': False
    }

    # Generators
    test_generator = DataGenerator(partition['test'], x_test, test_labels, **params)

    model = load_model(model_path, custom_objects={'auc': auc})

    preds = model.predict_generator(test_generator)

    true_labels = np.array(y_test_true).flatten()
    pred_labels = np.array([p[1] for p in preds])
    calculate_auc(true_labels, pred_labels)

In [5]:
data_dir = '../data'

x_test, y_test_true = load_data(data_dir, purpose='test', norm='macenko')

# indexes
test_id = np.arange(len(x_test))

# create a useful dictionary structures
partition = {}
partition['test'] = test_id
    
test_labels = {str(i) : y_test_true[i].flatten()[0] for i in test_id}

len(test_labels)

32768

## MODEL

In [11]:
# Define batch size.
batch_size = 128

# Parameters for generators
params = {
    'dim': (224, 224),
    'batch_size': batch_size,
    'n_classes': 2,
    'aug': False,
    'shuffle': False
}

# Generators
test_generator = DataGenerator(partition['test'], x_test, test_labels, **params)

dependencies = {
    'auc': auc
}

In [12]:
# network_filepath = '../Model/vgg16_model_bin_cross_arch_8_lr_0_001_inp96_first10ep/vgg16_model_bin_cross_arch_8_lr_0_001_inp96_epoch_10.h5' 
# network_filepath = '../Model/vgg16_model_bin_cross_arch_8_lr_0_001_inp96_10ep20ep/vgg16_model_bin_cross_arch_8_lr_0_001_inp96_epoch_5.h5' 
network_filepath = '../Model/vgg16_model_bin_cross_arch_8_lr_0_001_inp224/vgg16_model_bin_cross_arch_8_lr_0_001_inp224_epoch_1.h5' 

In [13]:
model = load_model(network_filepath, custom_objects=dependencies)

In [14]:
preds = model.predict_generator(test_generator)

### Archive

In [15]:
## vgg16_model_bin_cross_arch_8_lr_0_001_inp224_epoch_1
true_labels = np.array(y_test_true).flatten()
pred_labels = np.array([p[1] for p in preds])
calculate_auc(true_labels, pred_labels)

sklearn auc: 0.9093410523895604
tf auc: [0.0, 0.9093172]


In [37]:
## 10ep20ep/vgg16_model_bin_cross_arch_8_lr_0_001_inp96_epoch_5
true_labels = np.array(y_test_true).flatten()
pred_labels = np.array([p[1] for p in preds])
calculate_auc(true_labels, pred_labels)

sklearn auc: 0.8929814519587576
tf auc: [0.0, 0.8929175]
